In [ ]:
import pandas as pd
from collections import defaultdict

# Load the data into DataFrames with proper structure
metro_lines = pd.DataFrame([
    {"LineID": "M1", "Name": "Line 1 (Helwan-New Marg)", "Stations": ["12","1","3","F2","11"], "DailyPassengers": 1500000},
    {"LineID": "M2", "Name": "Line 2 (Shubra-Giza)", "Stations": ["11","F2","3","10","8"], "DailyPassengers": 1200000},
    {"LineID": "M3", "Name": "Line 3 (Airport-Imbaba)", "Stations": ["F1","5","2","3","9"], "DailyPassengers": 800000}
])

bus_routes = pd.DataFrame([
    {"RouteID": "B1", "Stops": ["1","3","6","9"], "BusesAssigned": 25, "DailyPassengers": 35000},
    {"RouteID": "B2", "Stops": ["7","15","8","10","3"], "BusesAssigned": 30, "DailyPassengers": 42000},
    {"RouteID": "B3", "Stops": ["2","5","F1"], "BusesAssigned": 20, "DailyPassengers": 28000},
    {"RouteID": "B4", "Stops": ["4","14","2","3"], "BusesAssigned": 22, "DailyPassengers": 31000},
    {"RouteID": "B5", "Stops": ["8","12","1"], "BusesAssigned": 18, "DailyPassengers": 25000},
    {"RouteID": "B6", "Stops": ["11","5","2"], "BusesAssigned": 24, "DailyPassengers": 33000},
    {"RouteID": "B7", "Stops": ["13","4","14"], "BusesAssigned": 15, "DailyPassengers": 21000},
    {"RouteID": "B8", "Stops": ["F7","15","7"], "BusesAssigned": 12, "DailyPassengers": 17000},
    {"RouteID": "B9", "Stops": ["1","8","10","9","6"], "BusesAssigned": 28, "DailyPassengers": 39000},
    {"RouteID": "B10", "Stops": ["F8","4","2","5"], "BusesAssigned": 20, "DailyPassengers": 28000}
])

demand = pd.DataFrame([
    {"FromID": "3", "ToID": "5", "DailyPassengers": 15000},
    {"FromID": "1", "ToID": "3", "DailyPassengers": 12000},
    {"FromID": "2", "ToID": "3", "DailyPassengers": 18000},
    {"FromID": "F2", "ToID": "11", "DailyPassengers": 25000},
    {"FromID": "F1", "ToID": "3", "DailyPassengers": 20000},
    {"FromID": "7", "ToID": "3", "DailyPassengers": 14000},
    {"FromID": "4", "ToID": "3", "DailyPassengers": 16000},
    {"FromID": "8", "ToID": "3", "DailyPassengers": 22000},
    {"FromID": "3", "ToID": "9", "DailyPassengers": 13000},
    {"FromID": "5", "ToID": "2", "DailyPassengers": 17000},
    {"FromID": "11", "ToID": "3", "DailyPassengers": 24000},
    {"FromID": "12", "ToID": "3", "DailyPassengers": 11000},
    {"FromID": "1", "ToID": "8", "DailyPassengers": 9000},
    {"FromID": "7", "ToID": "F7", "DailyPassengers": 18000},
    {"FromID": "4", "ToID": "F8", "DailyPassengers": 12000},
    {"FromID": "13", "ToID": "3", "DailyPassengers": 8000},
    {"FromID": "14", "ToID": "4", "DailyPassengers": 7000}
])

station_services = defaultdict(list)
for _, row in metro_lines.iterrows():
    for station in row["Stations"]:
        station_services[station].append(f"Metro: {row['LineID']}")

for _, row in bus_routes.iterrows():
    for station in row["Stops"]:
        station_services[station].append(f"Bus: {row['RouteID']}")



In [34]:
# Print DataFrame counts and basic info
print("\n=== DATASET OVERVIEW ===")
print(f"Metro Lines: {len(metro_lines)} lines")
print(f"Bus Routes: {len(bus_routes)} routes")
print(f"Demand Pairs: {len(demand)} station-to-station demands\n")




=== DATASET OVERVIEW ===
Metro Lines: 3 lines
Bus Routes: 10 routes
Demand Pairs: 17 station-to-station demands



In [ ]:
# Analysis functions (same as before)
def find_transfer_stations():
    transfer_stations = {k: v for k, v in station_services.items() if len(v) > 1}
    print("\n=== Key Transfer Stations ===")
    for station, services in sorted(transfer_stations.items(), key=lambda x: -len(x[1])):
        print(f"{station}: {len(services)} services ({', '.join(services)})")

find_transfer_stations()




=== Key Transfer Stations ===
3: 6 services (Metro: M1, Metro: M2, Metro: M3, Bus: B1, Bus: B2, Bus: B4)
2: 5 services (Metro: M3, Bus: B3, Bus: B4, Bus: B6, Bus: B10)
1: 4 services (Metro: M1, Bus: B1, Bus: B5, Bus: B9)
8: 4 services (Metro: M2, Bus: B2, Bus: B5, Bus: B9)
5: 4 services (Metro: M3, Bus: B3, Bus: B6, Bus: B10)
11: 3 services (Metro: M1, Metro: M2, Bus: B6)
10: 3 services (Metro: M2, Bus: B2, Bus: B9)
9: 3 services (Metro: M3, Bus: B1, Bus: B9)
4: 3 services (Bus: B4, Bus: B7, Bus: B10)
12: 2 services (Metro: M1, Bus: B5)
F2: 2 services (Metro: M1, Metro: M2)
F1: 2 services (Metro: M3, Bus: B3)
6: 2 services (Bus: B1, Bus: B9)
7: 2 services (Bus: B2, Bus: B8)
15: 2 services (Bus: B2, Bus: B8)
14: 2 services (Bus: B4, Bus: B7)


In [ ]:
def analyze_demand_coverage():
    print("\n=== Demand Coverage Analysis ===")
    uncovered_demand = []
    covered_high_demand = []
    
    for _, row in demand.iterrows():
        from_station, to_station, passengers = row["FromID"], row["ToID"], row["DailyPassengers"]
        covered = False
        
        # Check metro coverage
        for _, metro in metro_lines.iterrows():
            stations = metro["Stations"]
            if from_station in stations and to_station in stations:
                if stations.index(from_station) < stations.index(to_station):
                    covered = True
                    if passengers >= 9000:
                        covered_high_demand.append((from_station, to_station, passengers, "Metro"))
        
        # Check bus coverage
        for _, bus in bus_routes.iterrows():
            stops = bus["Stops"]
            if from_station in stops and to_station in stops:
                if stops.index(from_station) < stops.index(to_station):
                    covered = True
                    if passengers >= 9000:
                        covered_high_demand.append((from_station, to_station, passengers, "Bus"))
        
        if not covered and passengers > 5000:
            uncovered_demand.append((from_station, to_station, passengers))
    
    print("\nHigh-Demand Routes with Coverage:")
    for route in sorted(covered_high_demand, key=lambda x: -x[2]):
        print(f"{route[0]} → {route[1]}: {route[2]} passengers (covered by {route[3]})")
    
    print("\nUncovered High-Demand Routes:")
    for route in sorted(uncovered_demand, key=lambda x: -x[2]):
        print(f"{route[0]} → {route[1]}: {route[2]} passengers (no direct service)")
   
analyze_demand_coverage()   





=== Demand Coverage Analysis ===

High-Demand Routes with Coverage:
F2 → 11: 25000 passengers (covered by Metro)
11 → 3: 24000 passengers (covered by Metro)
8 → 3: 22000 passengers (covered by Bus)
F1 → 3: 20000 passengers (covered by Metro)
2 → 3: 18000 passengers (covered by Metro)
2 → 3: 18000 passengers (covered by Bus)
5 → 2: 17000 passengers (covered by Metro)
5 → 2: 17000 passengers (covered by Bus)
4 → 3: 16000 passengers (covered by Bus)
7 → 3: 14000 passengers (covered by Bus)
3 → 9: 13000 passengers (covered by Metro)
3 → 9: 13000 passengers (covered by Bus)
1 → 3: 12000 passengers (covered by Metro)
1 → 3: 12000 passengers (covered by Bus)
12 → 3: 11000 passengers (covered by Metro)
1 → 8: 9000 passengers (covered by Bus)

Uncovered High-Demand Routes:
7 → F7: 18000 passengers (no direct service)
3 → 5: 15000 passengers (no direct service)
4 → F8: 12000 passengers (no direct service)
13 → 3: 8000 passengers (no direct service)
14 → 4: 7000 passengers (no direct service)


In [40]:
def find_service_gaps():
    print("\n=== Service Gaps Analysis ===")
    all_stations = set(station_services.keys())
    
    # Stations with limited service
    limited_service = []
    for station, services in station_services.items():
        if len(services) == 1 and "Bus" in services[0]:
            limited_service.append((station, services[0]))
    
    print("\nStations with Only One Bus Service (Potential Gaps):")
    for station, service in limited_service:
        print(f"{station}: {service}")
    
    # High-demand stations with limited coverage
    high_demand_stations = set(demand["FromID"]).union(set(demand["ToID"]))
    critical_stations = []
    for station in high_demand_stations:
        if station not in all_stations:
            critical_stations.append(station)
        elif len(station_services[station]) == 1:
            critical_stations.append(station)
    
    print("\nCritical Stations Needing Better Coverage:")
    for station in critical_stations:
        if station in station_services:
            print(f"{station}: Only served by {station_services[station][0]}")
        else:
            print(f"{station}: Not served by any line!")
            
find_service_gaps()




=== Service Gaps Analysis ===

Stations with Only One Bus Service (Potential Gaps):
13: Bus: B7
F7: Bus: B8
F8: Bus: B10

Critical Stations Needing Better Coverage:
F7: Only served by Bus: B8
F8: Only served by Bus: B10
13: Only served by Bus: B7


In [52]:
import math
def analyze_capacity():
    print("\n=== CAPACITY ANALYSIS (REAL PEOPLE) ===")
    
    # Calculate passengers per bus (rounded up to whole people)
    bus_routes["PassengersPerBus"] = bus_routes["DailyPassengers"] / bus_routes["BusesAssigned"]
    bus_routes["PassengersPerBusCeil"] = bus_routes["PassengersPerBus"].apply(math.ceil)
    
    # Metro capacity analysis
    print("\nMETRO CAPACITY:")
    print(f"Total daily metro passengers: {math.ceil(metro_lines['DailyPassengers'].sum()):,} people")
    print("Passengers per metro line:")
    for _, row in metro_lines.iterrows():
        passengers = math.ceil(row['DailyPassengers'])
        print(f"- {row['LineID']}: {passengers:,} people ({passengers/len(row['Stations']):.0f} people/station)")
    
    # Bus capacity analysis
    print("\nBUS CAPACITY:")
    print(f"Total daily bus passengers: {math.ceil(bus_routes['DailyPassengers'].sum()):,} people")
    print(f"Total buses in operation: {bus_routes['BusesAssigned'].sum()}")
    
    # Bus crowding analysis
    print("\nBUS ROUTE CROWDING (people per bus):")
    for _, row in bus_routes.sort_values("PassengersPerBus", ascending=False).iterrows():
        passengers = math.ceil(row['DailyPassengers'])
        buses = row['BusesAssigned']
        per_bus = math.ceil(row['PassengersPerBus'])
        print(f"{row['RouteID']}: {per_bus} people/bus ({passengers:,} people / {buses} buses)")
    
analyze_capacity()


=== CAPACITY ANALYSIS (REAL PEOPLE) ===

METRO CAPACITY:
Total daily metro passengers: 3,500,000 people
Passengers per metro line:
- M1: 1,500,000 people (300000 people/station)
- M2: 1,200,000 people (240000 people/station)
- M3: 800,000 people (160000 people/station)

BUS CAPACITY:
Total daily bus passengers: 299,000 people
Total buses in operation: 214

BUS ROUTE CROWDING (people per bus):
B8: 1417 people/bus (17,000 people / 12 buses)
B4: 1410 people/bus (31,000 people / 22 buses)
B1: 1400 people/bus (35,000 people / 25 buses)
B2: 1400 people/bus (42,000 people / 30 buses)
B3: 1400 people/bus (28,000 people / 20 buses)
B7: 1400 people/bus (21,000 people / 15 buses)
B10: 1400 people/bus (28,000 people / 20 buses)
B9: 1393 people/bus (39,000 people / 28 buses)
B5: 1389 people/bus (25,000 people / 18 buses)
B6: 1375 people/bus (33,000 people / 24 buses)


In [130]:
import pandas as pd
from collections import defaultdict
import heapq

class TransportationData:
    def __init__(self):
        # Initialize all data containers
        self.traffic_flows = []
        self.metro_lines = []
        self.bus_routes = []
        self.transport_demand = []
        
        # Load all data
        self.load_traffic_data()
        self.load_metro_data()
        self.load_bus_data()
        self.load_demand_data()
        
        # Build network
        self.build_network()

    def load_traffic_data(self):
        """Load traffic flow data with explicit from/to separation"""
        traffic_records = [
            # Format: From, To, Morning, Afternoon, Evening, Night
            ('1', '3', 2800, 1500, 2600, 800),
            ('1', '8', 2200, 1200, 2100, 600),
            ('2', '3', 2700, 1400, 2500, 700),
            ('2', '5', 3000, 1600, 2800, 650),
            ('3', '5', 3200, 1700, 3100, 800),
            ('3', '6', 1800, 1400, 1900, 500),
            ('3', '9', 2400, 1300, 2200, 550),
            ('3', '10', 2300, 1200, 2100, 500),
            ('4', '2', 3600, 1800, 3300, 750),
            ('4', '14', 2800, 1600, 2600, 600),
            ('5', '11', 2900, 1500, 2700, 650),
            ('6', '9', 1700, 1300, 1800, 450),
            ('7', '8', 3200, 1700, 3000, 700),
            ('7', '15', 2800, 1500, 2600, 600),
            ('8', '10', 2000, 1100, 1900, 450),
            ('8', '12', 2400, 1300, 2200, 500),
            ('9', '10', 1800, 1200, 1700, 400),
            ('10', '11', 2200, 1300, 2100, 500),
            ('11', 'F2', 2100, 1200, 2000, 450),
            ('12', '1', 2600, 1400, 2400, 550),
            ('13', '4', 3800, 2000, 3500, 800),
            ('14', '13', 3600, 1900, 3300, 750),
            ('15', '7', 2800, 1500, 2600, 600),
            ('F1', '5', 3300, 2200, 3100, 1200),
            ('F1', '2', 3000, 2000, 2800, 1100),
            ('F2', '3', 1900, 1600, 1800, 900),
            ('F7', '15', 2600, 1500, 2400, 550),
            ('F8', '4', 2800, 1600, 2600, 600)
        ]
        
        self.traffic_flows = pd.DataFrame(traffic_records,
                                        columns=['From', 'To', 'Morning', 'Afternoon', 'Evening', 'Night'])
    def load_metro_data(self):
        """Load metro lines with station sequences"""
        metro_records = [
            ('M1', 'Line 1 (Helwan-New Marg)', ['12', '1', '3', 'F2', '11'], 1500000),
            ('M2', 'Line 2 (Shubra-Giza)', ['11', 'F2', '3', '10', '8'], 1200000),
            ('M3', 'Line 3 (Airport-Imbaba)', ['F1', '5', '2', '3', '9'], 800000)
        ]
        
        self.metro_lines = pd.DataFrame(metro_records,
                                      columns=['LineID', 'Name', 'Stations', 'DailyPassengers'])
    def load_bus_data(self):
        """Load bus routes with stop sequences"""
        bus_records = [
            ('B1', ['1', '3', '6', '9'], 25, 35000),
            ('B2', ['7', '15', '8', '10', '3'], 30, 42000),
            ('B3', ['2', '5', 'F1'], 20, 28000),
            ('B4', ['4', '14', '2', '3'], 22, 31000),
            ('B5', ['8', '12', '1'], 18, 25000),
            ('B6', ['11', '5', '2'], 24, 33000),
            ('B7', ['13', '4', '14'], 15, 21000),
            ('B8', ['F7', '15', '7'], 12, 17000),
            ('B9', ['1', '8', '10', '9', '6'], 28, 39000),
            ('B10', ['F8', '4', '2', '5'], 20, 28000)
        ]
        
        self.bus_routes = pd.DataFrame(bus_records,
                                     columns=['RouteID', 'Stops', 'BusesAssigned', 'DailyPassengers'])
        
    def load_demand_data(self):
        """Load origin-destination demand pairs"""
        demand_records = [
            ('3', '5', 15000),
            ('1', '3', 12000),
            ('2', '3', 18000),
            ('F2', '11', 25000),
            ('F1', '3', 20000),
            ('7', '3', 14000),
            ('4', '3', 16000),
            ('8', '3', 22000),
            ('3', '9', 13000),
            ('5', '2', 17000),
            ('11', '3', 24000),
            ('12', '3', 11000),
            ('1', '8', 9000),
            ('7', 'F7', 18000),
            ('4', 'F8', 12000),
            ('13', '3', 8000),
            ('14', '4', 7000)
        ]
        
        self.transport_demand = pd.DataFrame(demand_records,
                                           columns=['From', 'To', 'Passengers'])    
        
    def build_network(self):
        """Create a transport graph that preserves all connection types"""
        graph = defaultdict(lambda: defaultdict(dict))  # Changed to nested defaultdict
        
        # 1. Add traffic connections
        for _, row in self.traffic_flows.iterrows():
            from_node, to_node = row['From'], row['To']
            graph[from_node][to_node]['road'] = {  # Store under 'road' key
                'type': 'road',
                'traffic_data': {
                    'Morning': row['Morning'],
                    'Afternoon': row['Afternoon'],
                    'Evening': row['Evening'],
                    'Night': row['Night']
                }
            }
        
        # 2. Add metro connections
        for _, row in self.metro_lines.iterrows():
            stations = row['Stations']
            for i in range(len(stations)-1):
                graph[stations[i]][stations[i+1]]['metro'] = {  # Store under 'metro' key
                    'type': 'metro',
                    'line': row['LineID'],
                    'capacity': row['DailyPassengers'] / (len(stations)-1)
                }
        
        # 3. Add bus connections
        for _, row in self.bus_routes.iterrows():
            stops = row['Stops']
            for i in range(len(stops)-1):
                graph[stops[i]][stops[i+1]]['bus'] = {  # Store under 'bus' key
                    'type': 'bus',
                    'route': row['RouteID'],
                    'vehicles': row['BusesAssigned'],
                    'capacity': row['DailyPassengers'] / (len(stops)-1)
                }
        
        return graph
    
    def test_network(self):
        """Test the network building"""
        print("\n=== Testing Network Construction ===")
        network = self.build_network()
        
        # Test connections between two stations with path checking
        from_station = '1'  # Example start
        to_station = '3'   # Example destination
        
        print(f"\nChecking connections from {from_station} to {to_station}:")
        
        if from_station in network and to_station in network[from_station]:
            for conn_type, details in network[from_station][to_station].items():
                print(f"- {conn_type}: {details}")
        else:
            print(f"No path found from {from_station} to {to_station}")
        
        return network
    
    
    def find_routes_dp(self, start, end, time_of_day):
        """
        DP-based route finder using your network
        Returns: {path: [], time: minutes, transfers: int}
        """
        network = self.build_network()
        
        # DP table: {station: (min_time, path, transfers)}
        dp = {station: (float('inf'), [], 0) for station in network}
        dp[start] = (0, [start], 0)
        
        # Priority queue: (current_time, current_station, transfers)
        heap = [(0, start, 0)]
        
        while heap:
            current_time, current_station, transfers = heapq.heappop(heap)
            
            if current_station == end:
                return {
                    'path': dp[end][1],
                    'time': dp[end][0],
                    'transfers': dp[end][2]
                }
            
            for neighbor, connections in network[current_station].items():
                for conn_type, details in connections.items():
                    # Calculate time cost
                    if conn_type == 'road':
                        time_cost = details['traffic_data'][time_of_day] / 600  # 600 cars = 1 min
                    else:
                        time_cost = 3 if conn_type == 'metro' else 5  # Fixed transit times
                    
                    # Transfer penalty (add 5 mins if changing transport type)
                    last_leg_type = dp[current_station][1][-2] if len(dp[current_station][1]) > 1 else None
                    transfer_penalty = 5 if last_leg_type and last_leg_type != conn_type else 0
                    
                    new_time = current_time + time_cost + transfer_penalty
                    new_transfers = transfers + (1 if transfer_penalty > 0 else 0)
                    
                    # DP relaxation
                    if new_time < dp[neighbor][0]:
                        dp[neighbor] = (
                            new_time,
                            dp[current_station][1] + [neighbor],
                            new_transfers
                        )
                        heapq.heappush(heap, (new_time, neighbor, new_transfers))
        
        return {'error': 'No path found'}

    def get_human_readable_route(self, route):
        """Convert DP output to readable instructions"""
        if 'error' in route:
            return "No available route found"
        
        instructions = []
        path = route['path']
        
        for i in range(1, len(path)):
            from_st, to_st = path[i-1], path[i]
            conn = self.build_network()[from_st][to_st]
            
            if 'metro' in conn:
                instructions.append(f"Take {conn['metro']['line']} from {from_st} to {to_st} (3 mins)")
            elif 'bus' in conn:
                instructions.append(f"Take bus {conn['bus']['route']} from {from_st} to {to_st} (5 mins)")
            else:
                vehicles = conn['road']['traffic_data']['Morning']
                instructions.append(f"Travel by road from {from_st} to {to_st} ({vehicles} vehicles)")
        
        return (
            f"Optimal Route ({route['time']:.1f} mins, {route['transfers']} transfers):\n" +
            "\n".join(instructions)
        )
    
    def run_scheduler(self):
        """Interactive scheduling interface"""
        print("=== Public Transport Scheduler ===")
        while True:
            start = input("\nEnter start station (e.g., '1', 'F2'): ").strip()
            end = input("Enter destination station: ").strip()
            time = input("Enter time (Morning/Afternoon/Evening/Night): ").strip().capitalize()
            
            route = self.find_routes_dp(start, end, time)
            print("\n" + self.get_human_readable_route(route))
            
            if input("\nPlan another trip? (y/n): ").lower() != 'y':
                break
            

    def calculate_detailed_schedule(self, start, end, depart_time):
        """Generate timed schedule with exact departure/arrival times"""
        network = self.build_network()
        best_route = self.find_routes_dp(start, end, self.get_time_period(depart_time))
        
        # Convert depart_time to minutes since midnight (e.g., 7:30 AM = 450)
        current_time = self.time_to_minutes(depart_time)
        schedule = []
        
        path = best_route['path']
        for i in range(len(path)-1):
            from_st, to_st = path[i], path[i+1]
            conn = network[from_st][to_st]
            
            # Calculate segment time
            if 'metro' in conn:
                seg_time = 2 + len(conn['metro']['line'])*0.5  # M1/M2/M3 specific timing
            elif 'bus' in conn:
                seg_time = 5 + conn['bus']['traffic_data'].get(self.get_time_period(depart_time), 0)/800
            else:
                seg_time = 8  # Road travel
            
            departure = self.minutes_to_time(current_time)
            arrival = self.minutes_to_time(current_time + seg_time)
            schedule.append({
                'from': from_st,
                'to': to_st,
                'depart': departure,
                'arrive': arrival,
                'duration': seg_time,
                'type': 'metro' if 'metro' in conn else 'bus'
            })
            current_time += seg_time
        
        return schedule

    def generate_output(self, start, end, depart_time):
        """Produce your exact desired output format"""
        main_route = self.calculate_detailed_schedule(start, end, depart_time)
        alternatives = self.find_alternative_routes(start, end, depart_time)
        
        print(f"\n🟢 BEST OPTION ({self.total_route_time(main_route)} minutes total):")
        for seg in main_route:
            print(f"{seg['depart']} - Board {'Metro ' + seg['type'] if seg['type'] == 'metro' else 'Bus'} at Station {seg['from']}")
            print(f"{seg['arrive']} - Arrive Station {seg['to']} ({int(seg['duration'])} min ride)")
            if seg != main_route[-1]:
                next_depart = main_route[main_route.index(seg)+1]['depart']
                wait = (self.time_to_minutes(next_depart) - self.time_to_minutes(seg['arrive']))
                if wait > 0:
                    print(f"{seg['arrive']} - Board next transport ({wait} min wait)")
        
        if alternatives:
            print("\n🟡 Alternative Options:")
            for i, alt in enumerate(alternatives[:2]):  # Show max 2 alternatives
                print(f"\nOption {i+1} ({self.total_route_time(alt)} minutes):")
                for seg in alt:
                    print(f"{seg['depart']} - Take {'Metro' if seg['type'] == 'metro' else 'Bus'} from {seg['from']}")
                    print(f"{seg['arrive']} - Arrive {seg['to']} ({int(seg['duration'])} min)")

    # Helper methods
    def time_to_minutes(self, time_str):
        """Convert '7:30 AM' to minutes (450)"""
        time, period = time_str.split()
        hours, mins = map(int, time.split(':'))
        return hours*60 + mins + (720 if period == 'PM' else 0)

    def minutes_to_time(self, minutes):
        """Convert minutes to '8:15 AM' format"""
        return f"{minutes//60 % 12}:{minutes%60:02d} {'AM' if minutes < 720 else 'PM'}"

    def total_route_time(self, route):
        """Calculate total journey time"""
        return sum(seg['duration'] for seg in route)

    def get_time_period(self, time_str):
        """Convert time to Morning/Afternoon/Evening/Night"""
        hour = int(time_str.split(':')[0])
        if 5 <= hour < 12: return 'Morning'
        elif 12 <= hour < 17: return 'Afternoon'
        elif 17 <= hour < 21: return 'Evening'
        else: return 'Night'   
        
        
             

In [132]:
transport = TransportationData()
#network = transport.test_network()
#transport.run_scheduler()
transport.generate_output(start='1', end='5', depart_time='7:30 AM')

ValueError: Unknown format code 'd' for object of type 'float'